<a href="https://colab.research.google.com/github/jivemachine/GAN/blob/main/Simple_GAN_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

In [4]:
# importing the dataset from keras
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [5]:
# building the generator
size = 30 # size of images we are generating

generator = keras.models.Sequential([
      keras.layers.Dense(100, activation='selu', input_shape=[size]),
      keras.layers.Dense(150, activation='selu'),
      keras.layers.Dense(28 * 28, activation='sigmoid'),
      keras.layers.Reshape([28, 28])
])

In [6]:
# building the discriminator
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(150, activation='selu'),
    keras.layers.Dense(100, activation='selu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
# putting our simple gan together
gan = keras.models.Sequential([generator, discriminator])

In [8]:
# compiling the model
discriminator.compile(loss='binary_crossentropy', optimizer='rmsprop')
discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [9]:
# since the training loop is unusual we need to write a custom training loop for the model
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)
